<a href="https://colab.research.google.com/github/amuthalingeswaranbose/Amuthan_NLP_Learnings/blob/main/NLP_Spam_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
import nltk

nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package bcp47 to /root/nltk_data...
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package bllip_wsj_no_aux to /root/nltk_data...
       |   Unzipping models/bllip_wsj_no_aux.zip.
       | Downloading package book_grammars to /root/nltk_data...
       |   Un


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

# Load Dataset - Mount the drive

In [ ]:
!unzip /content/drive/MyDrive/NLP/sms+spam+collection.zip

Archive:  /content/drive/MyDrive/NLP/sms+spam+collection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [ ]:
import pandas as pd

df = pd.read_csv('/content/SMSSpamCollection', sep='\t', names=["label", "message"])
df.head(10)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


# Cleaning Data

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Object Initialization
po = PorterStemmer()
wordnet = WordNetLemmatizer()

In [ ]:
corpus = []

for i in range(len(df)):
  review = re.sub('^[a-zA-Z]', " ", df['message'][i]) # remove unwanted words and symbols except alphapets
  review = review.lower() # lower
  review = review.split() # split

  review = [po.stem(word) for word in review if word not in set(stopwords.words('english'))] # Stemming

  # review = [wordnet.lemmatize(word) for word in review if word not in set(stopwords.words('english'))] # Lemmatization

  review = " ".join(review)
  corpus.append(review)

corpus

['jurong point, crazy.. avail bugi n great world la e buffet... cine got amor wat...',
 'k lar... joke wif u oni...',
 "ree entri 2 wkli comp win fa cup final tkt 21st may 2005. text fa 87121 receiv entri question(std txt rate)t&c' appli 08452810075over18'",
 'dun say earli hor... u c alreadi say...',
 'ah think goe usf, live around though',
 "reemsg hey darl 3 week' word back! i'd like fun still? tb ok! xxx std chg send, £1.50 rcv",
 'ven brother like speak me. treat like aid patent.',
 "per request 'mell mell (oru minnaminungint nurungu vettam)' set callertun callers. press *9 copi friend callertun",
 'inner!! valu network custom select receivea £900 prize reward! claim call 09061701461. claim code kl341. valid 12 hour only.',
 'ad mobil 11 month more? u r entitl updat latest colour mobil camera free! call mobil updat co free 08002986030',
 "'m gonna home soon want talk stuff anymor tonight, k? i'v cri enough today.",
 'ix chanc win cash! 100 20,000 pound txt> csh11 send 87575. cost 

# Creating Bag of Words Model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)
x = cv.fit_transform(corpus).toarray()

# Creating TF-IDF Model

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# cv = TfidfVectorizer()
# x = cv.fit_transform(corpus).toarray()

In [ ]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
y = pd.get_dummies(df['label'])
y

,ham,spam
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0
...,...,...
5567,0,1
5568,1,0
5569,1,0
5570,1,0


In [ ]:
y = y.iloc[:, 1].values
y

array([0, 0, 1, ..., 0, 0, 0], dtype=uint8)

# Train and Test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)
spam_detect_model

MultinomialNB()

In [ ]:
y_pred = spam_detect_model.predict(X_test)

In [ ]:
y_pred

array([0, 1, 0, ..., 0, 1, 0], dtype=uint8)

# Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[945,  10],
       [  6, 154]])

# Accuracy

In [ ]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred)
acc

0.9856502242152466